# Optibook Manual

This notebook contains examples for **all interactions** you can do with optibook (that means inserting orders, getting your positions, etc.). You should use this notebook as a reference/documenation for the system later on when you write more extensive algorithms.

First we do some setup and import the optibook client, which is used to connect to the optibook exchange. If everything is setup correctly, you should see the line "Setup was successful." being printed.

In [1]:
from optibook.synchronous_client import Exchange

import logging
logger = logging.getLogger('client')
logger.setLevel('ERROR')

print("Setup was successful.")

Setup was successful.


### Define Your Instrument

An instrument is the term we use for the 'thing' that we are trading. As such, if we are trading BMW stocks, we would call those an instrument. However, BMW options would be a different instrument. 

This field determines which instrument we are trading. By changing it, you will insert trades for a different instrument. You can see all available instruments by looking at the dropdown menu labeled "Instruments" on the visualizer.

In [2]:
instrument_id = 'PHILIPS_A'

### Connect to Exchange

In [3]:
e = Exchange()
a = e.connect()

# you can also define host/user/pass yourself
# when not defined, it is taken from ~/.optibook file if it exists
# if that file does not exists, an error is thrown

#e = Exchange(host='host-to-connect-to')
#a = e.connect(username='your-username', password='your-password')


2021-01-24 14:25:17,167 [asyncio   ] [MainThread  ] Using selector: EpollSelector


## Outstanding Orders, Trades, Current Positions and PnL

In [31]:
# Returns all currently outstanding orders
orders = e.get_outstanding_orders(instrument_id)
for o in orders.values():
    print(o)

In [32]:
# Returns all trades you have done since the last time this function was called
trades = e.poll_new_trades(instrument_id)
for t in trades:
    print(f"[TRADED {t.instrument_id}] price({t.price}), volume({t.volume}), side({t.side})")

In [100]:
# Returns all trades you have done since since the instantiation of the Exchange
trades = e.get_trade_history(instrument_id)
for t in trades:
    print(f"[TRADED {t.instrument_id}] price({t.price}), volume({t.volume}), side({t.side})")

[TRADED PHILIPS_A] price(85.10000000000001), volume(1), side(bid)
[TRADED PHILIPS_A] price(83.80000000000001), volume(1), side(ask)
[TRADED PHILIPS_A] price(85.9), volume(1), side(ask)


In [96]:
# Returns all current positions
positions = e.get_positions()
for p in positions:
    print(p, positions[p])

PHILIPS_A -240
PHILIPS_B 398


In [34]:
# Returns all current positions with cash invested
positions = e.get_positions_and_cash()
for p in positions:
    print(p, positions[p])

PHILIPS_A {'volume': 0, 'cash': 509.2999999999938}
PHILIPS_B {'volume': -509, 'cash': 34963.29999999999}


In [104]:
# Returns Current PnL based on last Traded Price
pnl = e.get_pnl()
print(pnl)

168.20000000000067


In [99]:
from IPython.display import clear_output
import time
class order_level:
    def __init__(self, bidorask, price, vol):
        if bidorask == "bid":
            self.bid_volume = vol
            self.ask_volume = ""
        if bidorask == "ask":
            self.bid_volume = ""
            self.ask_volume = vol
            
        self.price_level = price

def weighted_mid():
    book = e.get_last_price_book(instrument_id)
    bidslist = []
    asklist = []
    bids_vol = 0
    asks_vol = 0

    try:
        for bid in book.bids:
            bids_vol += bid.volume
            bid_each = order_level("bid",round(bid.price,1),bid.volume)
            bidslist.append(bid_each)

        for ask in book.asks:
            asks_vol += ask.volume
            ask_each = order_level("ask",round(ask.price,1),ask.volume)
            asklist.append(ask_each)

        bid_weight = bids_vol/(bids_vol+asks_vol)
        ask_weight = asks_vol/(bids_vol+asks_vol)
        weighted_mid = (bidslist[0].price_level)*bid_weight+(asklist[0].price_level)*ask_weight

        return round(weighted_mid,1)
    
    except:
        return "error"

while True:
    clear_output(wait=True)
    if weighted_mid() == "error":
        continue
    else:   
        print("Weighted mid:", weighted_mid())
        time.sleep(0.01)


Weighted mid: 73.2


KeyboardInterrupt: 

## Order Book and Public Trade Ticks

In [101]:
class order_level:
    def __init__(self, bidorask, price, vol):
        if bidorask == "bid":
            self.bid_volume = vol
            self.ask_volume = ""
        if bidorask == "ask":
            self.bid_volume = ""
            self.ask_volume = vol
        self.price_level = price
        
        
book = e.get_last_price_book(instrument_id)
processed_order_book = []
print("bids: ", book.bids)
print("-------")
for bid in book.bids:
#     bid_dic = {round(bid.price,1):bid.volume}
#     total['bids'].append(bid_dic)
#     print("bids: ", round(bid.price, 1))
    bid_each = order_level("bid",round(bid.price,1),bid.volume)
    processed_order_book.append(bid_each)
    
print("asks: ", book.asks)
print("-------")
for ask in book.asks:
#     ask_dic = {round(ask.price,1):ask.volume}
#     total['asks'].append(ask_dic)
#     print("asks:", round(ask.price, 1))
    ask_each = order_level("ask",round(ask.price,1),ask.volume)
    processed_order_book.append(ask_each)

print("bid | price | ask")

for level in processed_order_book:
  print(f"{level.bid_volume} | {level.price_level} | {level.ask_volume}")


    


bids:  [[price_volume] price=73.8, volume=40, [price_volume] price=73.7, volume=200]
-------
asks:  [[price_volume] price=73.9, volume=50, [price_volume] price=74.3, volume=200]
-------
bid | price | ask
40 | 73.8 | 
200 | 73.7 | 
 | 73.9 | 50
 | 74.3 | 200


In [76]:
# Returns all public tradeticks since the last time this function was called
tradeticks = e.poll_new_trade_ticks(instrument_id)
for t in tradeticks:
    print(f"[{t.instrument_id}] price({t.price}), volume({t.volume}), aggressor_side({t.aggressor_side}), buyer({t.buyer}), seller({t.seller})")
print(len())    

AssertionError: Cannot call function until connected. Call connect() first

In [77]:
# Returns all public tradeticks since the instantiation of the Exchange
tradeticks = e.get_trade_tick_history(instrument_id)
for t in tradeticks:
    print(t)
print(len(tradeticks)) 

AssertionError: Cannot call function until connected. Call connect() first

In [ ]:
# See all your outstanding orders
outstanding = e.get_outstanding_orders(instrument_id)
for o in outstanding.values():
    print(f"Outstanding order: order_id({o.order_id}), instrument_id({o.instrument_id}), price({o.price}), volume({o.volume}), side({o.side})")

## Inserting and Deleting Orders

In [102]:
# Insert bid limit order - This trades against any current orders, and any remainders become new resting orders in the book
# Use this to buy.
result = e.insert_order(instrument_id, price=74.5, volume=1, side='bid', order_type='limit')
print(f"Order Id: {result}")

Order Id: 600048


In [56]:
# Insert ask limit order - This trades against any current orders, and any remainders become new resting orders in the book
# Use this to sell.
result = e.insert_order(instrument_id, price=83, volume=1, side='ask', order_type='limit')
print(f"Order Id: {result}")

Order Id: 548510


In [ ]:
# Insert bid IOC - This can trade against any resting volume but does not remain in the book
# Use this to buy.
result = e.insert_order(instrument_id, price=445.0, volume=1, side='bid', order_type='ioc')
print(f"Order Id: {result}")

In [ ]:
# Insert ask IOC - This can trade against any resting volume but does not remain in the book
# Use this to sell.
result = e.insert_order(instrument_id, price=430.0, volume=1, side='ask', order_type='ioc')
print(f"Order Id: {result}")

In [ ]:
# Attempt to delete inserted order by order_id
order_id = 4
result = e.delete_order(instrument_id, order_id=order_id)
print()
print(f"Deleted order id {order_id}: {result}")

In [ ]:
# Change volume for existing order
order_id = 5
new_volume = 16
result = e.amend_order(instrument_id, order_id=order_id, volume=new_volume)
print(f"Changed volume for order id {order_id} to {new_volume} lots: {result}")

In [ ]:
# Delete all outstanding orders
outstanding = e.get_outstanding_orders(instrument_id)
for o in outstanding.values():
    result = e.delete_order(instrument_id, order_id=o.order_id)
    print(f"Deleted order id {o.order_id}: {result}")

## 'Hack' Out of Positions

In [5]:
# Get out of all positions you are currently holding, regarless of the loss involved. That means selling whatever
# you are long, and buying-back whatever you are short. Be sure you know what you are doing when you use this logic.
print(e.get_positions())
for s, p in e.get_positions().items():
    if p > 0:
        e.insert_order(s, price=1, volume=p, side='ask', order_type='ioc')
    elif p < 0:
        e.insert_order(s, price=10000, volume=-p, side='bid', order_type='ioc')  
print(e.get_positions())

{'PHILIPS_A': 0, 'PHILIPS_B': 0}
{'PHILIPS_A': 0, 'PHILIPS_B': 0}


2021-01-24 14:26:41,925 [client    ] [Thread-4    ] Forcing a disconnect due to an error: Closing connection because someone else logged in with the same credentials. Only one session may be active at the same time.
